In [1]:
import pandas as pd

# Load the CSV file
file_path = 'TQQQ-15second-2024-08.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
data.head(), data.columns


(                   timestamp   open   high    low  close  volume     vwap
 0  2024-08-01 04:00:00-04:00  69.58  70.00  69.50  69.53    5492  69.6862
 1  2024-08-01 04:00:15-04:00  69.48  69.55  69.48  69.54    1099  69.5325
 2  2024-08-01 04:00:30-04:00  69.60  69.62  69.55  69.55    5276  69.5578
 3  2024-08-01 04:00:45-04:00  69.53  69.54  69.50  69.50    3699  69.5135
 4  2024-08-01 04:01:00-04:00  69.52  69.53  69.44  69.45    7907  69.4827,
 Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'vwap'], dtype='object'))

In [3]:
import numpy as np
import talib

# Ensure the data is sorted by timestamp
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data.sort_values(by='timestamp')

# Calculating RSI
data['RSI'] = talib.RSI(data['close'], timeperiod=14)

# Calculating EMA Fast and Slow
data['EMA_Fast'] = talib.EMA(data['close'], timeperiod=3)
data['EMA_Slow'] = talib.EMA(data['close'], timeperiod=9)

# MACD calculation
data['MACD'], data['MACD_Signal'], _ = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Calculate envelope
envelope_length = 21
envelope_percent = 0.3 / 100
data['Envelope_Upper'] = talib.EMA(data['close'], timeperiod=envelope_length) * (1 + envelope_percent)
data['Envelope_Lower'] = talib.EMA(data['close'], timeperiod=envelope_length) * (1 - envelope_percent)

# Prepare the data for further conditions
data[['timestamp', 'RSI', 'EMA_Fast', 'EMA_Slow', 'MACD', 'MACD_Signal', 'Envelope_Upper', 'Envelope_Lower']].head()


ModuleNotFoundError: No module named 'talib'

In [ ]:
# Implementing custom EMA calculation
def ema(series, period):
    return series.ewm(span=period, adjust=False).mean()

# Implementing custom RSI calculation
def rsi(series, period=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculating RSI, EMA Fast and Slow
data['RSI'] = rsi(data['close'], period=14)
data['EMA_Fast'] = ema(data['close'], period=3)
data['EMA_Slow'] = ema(data['close'], period=9)

# Implementing custom MACD calculation
def macd(series, fastperiod=12, slowperiod=26, signalperiod=9):
    ema_fast = ema(series, fastperiod)
    ema_slow = ema(series, slowperiod)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signalperiod)
    return macd_line, signal_line

data['MACD'], data['MACD_Signal'] = macd(data['close'])

# Calculate envelope
envelope_length = 21
envelope_percent = 0.3 / 100
data['Envelope_Upper'] = ema(data['close'], period=envelope_length) * (1 + envelope_percent)
data['Envelope_Lower'] = ema(data['close'], period=envelope_length) * (1 - envelope_percent)

# Displaying the calculated columns
data[['timestamp', 'RSI', 'EMA_Fast', 'EMA_Slow', 'MACD', 'MACD_Signal', 'Envelope_Upper', 'Envelope_Lower']].head()


In [ ]:
# Initialize columns for scores
data['score_ema'] = 0
data['score_macd'] = 0
data['score_rsi30'] = 0
data['score_envelope'] = 0
data['score_bullishEng'] = 0
data['score_bullCond'] = 0

data['sell_score_ema'] = 0
data['sell_score_macd'] = 0
data['sell_score_rsi70'] = 0
data['sell_score_envelope'] = 0
data['sell_score_bearCond'] = 0

# Variables to track scores
max_buy_score = 700
max_sell_score = 600

# Define the conditions and score calculations for each row
for i in range(1, len(data)):
    # Buy conditions
    if data['EMA_Fast'].iloc[i] > data['EMA_Slow'].iloc[i] and data['EMA_Fast'].iloc[i-1] <= data['EMA_Slow'].iloc[i-1]:
        data.at[i, 'score_ema'] = 100
    else:
        data.at[i, 'score_ema'] = max(0, data.at[i-1, 'score_ema'] - 10)
    
    if data['MACD'].iloc[i] > data['MACD_Signal'].iloc[i] and data['MACD'].iloc[i] < 0 and data['MACD'].iloc[i-1] <= data['MACD_Signal'].iloc[i-1]:
        data.at[i, 'score_macd'] = 100
    else:
        data.at[i, 'score_macd'] = max(0, data.at[i-1, 'score_macd'] - 10)
    
    if data['RSI'].iloc[i] > 30 and data['RSI'].iloc[i-1] <= 30:
        data.at[i, 'score_rsi30'] = 100
    else:
        data.at[i, 'score_rsi30'] = max(0, data.at[i-1, 'score_rsi30'] - 10)
    
    if data['EMA_Fast'].iloc[i] > data['Envelope_Lower'].iloc[i] and data['EMA_Fast'].iloc[i-1] <= data['Envelope_Lower'].iloc[i-1]:
        data.at[i, 'score_envelope'] = 100
    else:
        data.at[i, 'score_envelope'] = max(0, data.at[i-1, 'score_envelope'] - 10)

    # Sell conditions
    if data['EMA_Fast'].iloc[i] < data['EMA_Slow'].iloc[i] and data['EMA_Fast'].iloc[i-1] >= data['EMA_Slow'].iloc[i-1]:
        data.at[i, 'sell_score_ema'] = 100
    else:
        data.at[i, 'sell_score_ema'] = max(0, data.at[i-1, 'sell_score_ema'] - 10)
    
    if data['MACD'].iloc[i] < data['MACD_Signal'].iloc[i] and data['MACD'].iloc[i] > 0 and data['MACD'].iloc[i-1] >= data['MACD_Signal'].iloc[i-1]:
        data.at[i, 'sell_score_macd'] = 100
    else:
        data.at[i, 'sell_score_macd'] = max(0, data.at[i-1, 'sell_score_macd'] - 10)
    
    if data['RSI'].iloc[i] < 70 and data['RSI'].iloc[i-1] >= 70:
        data.at[i, 'sell_score_rsi70'] = 100
    else:
        data.at[i, 'sell_score_rsi70'] = max(0, data.at[i-1, 'sell_score_rsi70'] - 10)
    
    if data['EMA_Fast'].iloc[i] < data['Envelope_Upper'].iloc[i] and data['EMA_Fast'].iloc[i-1] >= data['Envelope_Upper'].iloc[i-1]:
        data.at[i, 'sell_score_envelope'] = 100
    else:
        data.at[i, 'sell_score_envelope'] = max(0, data.at[i-1, 'sell_score_envelope'] - 10)

# Calculate total scores and percentages
data['score'] = data['score_ema'] + data['score_macd'] + data['score_rsi30'] + data['score_envelope'] + data['score_bullCond']
data['sell_score'] = data['sell_score_ema'] + data['sell_score_macd'] + data['sell_score_rsi70'] + data['sell_score_envelope'] + data['sell_score_bearCond']

data['total_percent'] = (data['score'] / max_buy_score) * 100
data['total_sell_percent'] = (data['sell_score'] / max_sell_score) * 100

# Display relevant columns for buy/sell signals
data[['timestamp', 'score', 'sell_score', 'total_percent', 'total_sell_percent']].tail()
